<a href="https://colab.research.google.com/github/t-willi/Simula/blob/main/CleanAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer 
import glob
import torch.optim as optim
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
from tqdm.auto import tqdm
import requests
import zipfile
from pathlib import Path
!pip install ecg_plot
!pip install wandb -Uq
!pip install -Uqq ipdb
import ipdb


if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.4 MB/s 
     |████████████████████████████████| 182 kB 53.7 MB/s 
     |████████████████████████████████| 166 kB 57.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 162 kB 55.2 MB/s 
     |████████████████████████████████| 162 kB 54.9 MB/s 
     |████████████████████████████████| 158 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 63.1 MB/s 
     |████████████████████████████████| 157 kB 54.3 MB/s 
     |████████████████████████████████| 157 kB 45.8 MB/s 
     |████████████████████████████████| 157 kB 47.6 MB/s 
     |████████████████████████████████| 157 kB 66.1 MB/s 
     |████████████████████████████████| 157 kB 56.9 MB/s 
     |████████████████████████████████| 157 kB 56.8 MB/s 
     |████████████████████████████████| 156 kB 55.7 MB/s 
     |████████████████████████████

In [ ]:
%pdb off


Automatic pdb calling has been turned OFF


In [2]:
def request(path=None,name=None):
  import requests
  from pathlib import Path
  request = requests.get(path)
  name=name+".py"
  with open(name,"wb") as f:
    f.write(request.content)

In [3]:
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [26]:
def unzip(path=None):
  # Setup path to data folder
  data_path = Path("data/")
  train_path = data_path / "train_data"
  # If the image folder doesn't exist, download it and prepare it... 
  if train_path.is_dir():
      print(f"{train_path} directory exists.")
  else:
      print(f"Did not find {train_path} directory, creating one...")
      train_path.mkdir(parents=True, exist_ok=True)
      # Unzip pizza, steak, sushi data
      with zipfile.ZipFile("/content/artifacts/ecg_25000.zip:v0/ecg_25000.zip", "r") as zip_ref:
          print(f"Unzipping data to folder...") 
          zip_ref.extractall(train_path)

wandb: Downloading large artifact ecg_25000.zip:v0, 1383.92MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.1


data/train_data directory exists.
Unzipping data to folder...


In [5]:
## Load Custom dataloader
Dataloader_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/Dataloader.py"
request(Dataloader_git_dir,"Dataloader")
from Dataloader import Custom_dataset as CD

def make_loader(dataset,batch_size):
  loader = DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True
                      )
  return loader

In [22]:
#Download and instantialize model
model_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/Networks.py"
request(model_git_dir,"Networks")
from Networks import ECG_AE_v1
model=ECG_AE_v1()
model

ECG_AE_v1(
  (encoder): Sequential(
    (0): Linear(in_features=5000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=20, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=35000, bias=True)
  )
)

In [ ]:
#download prediction generator
# get_pred_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_predictions.py"
# request(get_pred_git_dir,"get_predictions")
# from get_predictions import get_pred
# #download ECG plotter
# plt_ECG_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/plot_ECG.py"
# request(plt_ECG_git_dir,"plot_ECG")
# from plot_ECG import plotECG
#input,output=get_pred(dataset=val_dataset,model=model)

In [23]:
# 🐝 initialise a wandb run
config = dict(
    epochs=10,
    batch_size=128,
    learning_rate=0.001,)


In [30]:
def model_pipeline(hyperparameters,model=model):
    # tell wandb to get started
    artifact = wandb.init(project="pytorch-demo", config=hyperparameters).use_artifact('ecg_simula/setup_weights and biases/ecg_25000.zip:v0', type='raw_data')
    artifact_dir = artifact.download()
    unzip()
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config
    # make the model, data, and optimization problem
    train_loader, val_loader, criterion, optimizer = make(config)
    # and use them to train the model
    train(model, train_loader,val_loader, criterion, optimizer, config)
    return model

In [33]:
def make(config):
    # Make the data
    data_dir="/content/data/train_data"
    train_dataset = CD(data_dir=data_dir,split=True,target="train")
    val_dataset = CD(data_dir=data_dir,split=True,target="val")
    train_loader = make_loader(train_dataset, batch_size=config.batch_size)
    val_loader = make_loader(val_dataset, batch_size=config.batch_size)

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    
    return train_loader, val_loader, criterion, optimizer

In [19]:
def train(model, train_loader,val_loader, criterion, optimizer, config):
  # Tell wandb to watch what the model gets up to: gradients, weights, and more!
  wandb.watch(model, criterion, log="all")
  for epoch in tqdm(range(config.epochs)):
    train_loss=0
    for batch,(X,y) in tqdm(enumerate(train_loader)):
      # Forward pass ➡
      X, y = X.to(device), y.to(device) 
      model.train()
      output=model(X)
      output=torch.reshape(output,(config.batch_size, 1, 7, 5000))
      loss = criterion(output,y)
      train_loss += loss
      # Backward pass ⬅
      optimizer.zero_grad()
      loss.backward()
      # Step with optimizer
      optimizer.step()
    #average loss per batch
    train_loss /= len(train_loader)

    val_loss = 0
    model.eval()
    with torch.inference_mode():
      for batch,(X,y) in enumerate(val_loader):
        #print("doing test loop")
        X, y = X.to(device), y.to(device)
        val_pred = model(X)
        val_pred=torch.reshape(val_pred,(128, 1, 7, 5000))
        loss=criterion(val_pred,y)
        val_loss += loss
      val_loss /= len(val_loader)  
      wandb.log({"train_loss": train_loss, 
                 "val_loss": val_loss})
  if epoch % 25==0:
    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model,"model.onnx")
    wandb.save("model.onnx")


In [ ]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config)

wandb: Downloading large artifact ecg_25000.zip:v0, 1383.92MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.1


data/train_data directory exists.


  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]